In [ ]:
import pandas as pd
from glob import glob
from pathlib import Path


rg_raw_data_dir = "data/7-23-25-scrape"
rg_fps = glob(f"{rg_raw_data_dir}/*.csv")

# read + show an example set of raw vals for a rain-gauge
ex_fp = rg_fps[0]
ex_df = pd.read_csv(ex_fp)
ex_df.head()

,Date,Time,Value
0,07/23/2025,13:00:00,0.63
1,07/23/2025,12:00:00,0.63
2,07/23/2025,11:00:00,0.63
3,07/23/2025,10:00:00,0.63
4,07/23/2025,09:00:00,0.63


### 1. Grouping rain gauge data into one table
---
- Aggregate rain gauge raw data (i.e., bucket tips) into a ***sparse*** table with timestep index $\mathcal{T}_{CCRFCD}$
    - Why is this table "sparse"?
        - Timesteps $u \in \mathcal{T}_{CCRFCD}$ are not uniformly spaced
        - Each timestep $u$ (in theory) corresponds to **one** datapoint for **one** rain gauge
    - Sort `datetime-local` column 

| gauge-idx | datetime-local | utc-offset | datetime-utc | val |
| :---: | :---: | :---: | :---: | :---: |
| 4 | 2025-03-03-07:03 | -7 | 2025-03-03-14:03 | 3.5 |
| 4 | 2025-03-03-07:35 | -7 | 2025-03-03-14:35 | 3.6 |

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo


# PST/PDT
las_vegas_tz = ZoneInfo("America/Los_Angeles")


def gen_rg_df(df: pd.DataFrame, gauge_idx: int) -> pd.DataFrame:

    local_dts = []
    utc_dts = []

    date_arr = df['Date']; time_arr = df['Time']
    concat_str_arr = date_arr + ' ' + time_arr
    
    for dt_str in concat_str_arr:
        
        # local_dt
        naive_dt = datetime.strptime(dt_str, "%m/%d/%Y %H:%M:%S")
        local_dt = naive_dt.replace(tzinfo=las_vegas_tz)

        # local -> UTC
        utc_dt = local_dt.astimezone(ZoneInfo("UTC"))

        local_dts.append(local_dt)
        utc_dts.append(utc_dt)

    instant_rain_acc = df["Value"]

    gauge_df = pd.DataFrame({
        'gauge_idx': [gauge_idx] * len(local_dts), # repeat gauge_id for number of rows
        'local_time': local_dts,
        'utc_time': utc_dts,
        'gauge_acc_in': instant_rain_acc,
    })

    return gauge_df

In [ ]:
from tqdm import tqdm


master_df: pd.DataFrame | None = None

# iterate through CCRFCD rain gauge raw data `.csv` files
# open each dataframe; calculate utc datetime, append df to `master_df`
for fp in tqdm(rg_fps):

    # hackey way of getting the rain gauge's idx from the fp
    rg_idx = int(fp.split("_")[-1: ][0].strip(".csv"))

    _df = pd.read_csv(fp)
    rg_df = gen_rg_df(_df, rg_idx)

    if master_df is None:
        master_df = rg_df
    else:
        
        # vertical concatination
        _temp_df = pd.concat([master_df, rg_df], axis=0)
        master_df = _temp_df

  0%|          | 0/231 [00:00<?, ?it/s]

 80%|████████  | 185/231 [00:10<00:02, 17.48it/s]/tmp/ipykernel_3478149/549790837.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  _temp_df = pd.concat([master_df, rg_df], axis=0)
100%|██████████| 231/231 [00:12<00:00, 18.26it/s]


In [ ]:
# hmm... there are some unrealistically large/small values in our dataset
master_df.sort_values("gauge_acc_in", ascending=False)

,gauge_idx,local_time,utc_time,gauge_acc_in
411,4651.0,2025-07-17 22:37:53-07:00,2025-07-18 05:37:53+00:00,416.0
3562,4139.0,2025-07-02 18:12:00-07:00,2025-07-03 01:12:00+00:00,359.0
3589,4139.0,2025-07-02 17:15:20-07:00,2025-07-03 00:15:20+00:00,359.0
3613,4139.0,2025-07-02 16:52:20-07:00,2025-07-02 23:52:20+00:00,359.0
3580,4139.0,2025-07-02 17:50:40-07:00,2025-07-03 00:50:40+00:00,359.0
...,...,...,...,...
4277,4702.0,2025-04-05 19:05:41-07:00,2025-04-06 02:05:41+00:00,-9.0
704,4702.0,2025-07-07 02:50:52-07:00,2025-07-07 09:50:52+00:00,-9.0
3626,4702.0,2025-04-25 14:21:57-07:00,2025-04-25 21:21:57+00:00,-10.0
1692,4702.0,2025-06-08 14:23:03-07:00,2025-06-08 21:23:03+00:00,-11.0


### 1.1 Removing non-rain gauges

- Some of these sensors arn't rain gauges! For example, `4651.0` is an annemometer, explaining the unusally high values.
- We'll simply mask out any sensor that isn't a rain gauge.

In [ ]:
sensor_metadata = pd.read_csv("data/ccrfcd_rain_gauge_metadata.csv")

# every valid rain gauge contains the substring "(Rain)" in it's description
mask = sensor_metadata["old_name"].str.contains(r"\(Rain\)", na=False)

# set of valid rg ids
valid_rain_gauges = set(sensor_metadata[mask]["station_id"])

# mask out any row that doesn't contain a valid rain gauge id
valid_rain_gauges_mask = [idx in valid_rain_gauges for idx in master_df['gauge_idx']]

# let's check our highest/lowest precip. accumulation values again
master_df_cleaned = master_df[valid_rain_gauges_mask].sort_values("gauge_acc_in", ascending=False)
master_df_cleaned

,gauge_idx,local_time,utc_time,gauge_acc_in
2050,4359.0,2023-09-28 18:34:42-07:00,2023-09-29 01:34:42+00:00,20.04
2047,4359.0,2023-09-30 06:34:45-07:00,2023-09-30 13:34:45+00:00,20.04
2052,4359.0,2023-09-28 03:37:52-07:00,2023-09-28 10:37:52+00:00,20.04
2046,4359.0,2023-09-30 18:34:46-07:00,2023-10-01 01:34:46+00:00,20.04
2048,4359.0,2023-09-29 18:34:44-07:00,2023-09-30 01:34:44+00:00,20.04
...,...,...,...,...
2888,4174.0,2022-11-06 08:23:31-08:00,2022-11-06 16:23:31+00:00,0.00
2887,4174.0,2022-11-06 16:23:32-08:00,2022-11-07 00:23:32+00:00,0.00
2886,4174.0,2022-11-07 00:23:31-08:00,2022-11-07 08:23:31+00:00,0.00
2885,4174.0,2022-11-07 08:23:31-08:00,2022-11-07 16:23:31+00:00,0.00


In [ ]:
checkpoint_one_fp = "data/__checkpoints__/ds_ckpt_1.csv"
master_df_cleaned.sort_values("local_time").to_csv(checkpoint_one_fp)

### 2. Selecting which MRMS 1H QPE files to download
---

- Construct $\mathcal{T}_{MRMS}$; i.e., determine which MRMS 1H-QPE timesteps to download
    - LV valley: (35.8, 36.4 / -115.4, -114.8)
    - 1 1km $\times$ 1km grid-cell in the LV valley >= 0.25 in. 1H QPE in a 24H period (i.e., 00:00-23:59 UTC)
- Next steps
    - All data is downloaded to: `__temp/*`

In [6]:
import pandas as pd


checkpoint_one_fp = "data/__checkpoints__/ds_ckpt_1.csv"
ckpt_one_df = pd.read_csv(checkpoint_one_fp, )
ckpt_one_df.head()

,Unnamed: 0,gauge_idx,local_time,utc_time,gauge_acc_in
0,3753,4054.0,2021-01-01 08:00:25-08:00,2021-01-01 16:00:25+00:00,0.00
1,4652,4394.0,2021-01-01 08:01:13-08:00,2021-01-01 16:01:13+00:00,0.00
2,3263,4309.0,2021-01-01 08:01:44-08:00,2021-01-01 16:01:44+00:00,0.04
3,3643,8.0,2021-01-01 08:05:16-08:00,2021-01-01 16:05:16+00:00,0.00
4,3603,4574.0,2021-01-01 08:09:26-08:00,2021-01-01 16:09:26+00:00,0.00


### 3. Generate a ***dense*** table from `1` with a timestep index of $\mathcal{T}_{MRMS}$
---
- > $\forall_{t \in \mathcal{T}_{MRMS}} \forall_{k \in \mathcal{I}} \exists $ *row in datatable*

- 3a. $\forall_{t \in \mathcal{T}_{MRMS}} \forall_{k \in \mathcal{I}}$ ...
    - Lookup rows for $k$ between $[t_{start}, t_{end}]$
    - Calculate the *sum* of **positive** differences only
        - e.g., `[1, 1.2, 0.0, 0.3] -> [NaN, 0.2, NaN, 0.3] -> 0.5`
        - Rain gauges occasionally *reset*; negative rainfall amounts are impossible

| gauge-idx | start-datetime-utc | end-datetime-utc | gauge-1h-acc |
| :---: | :---: | :---: | :---: |
| 4 | 2025-03-03-14:00 | 2025-03-03-15:00 | 0.1 |
| 4 | 2025-03-03-14:02 | 2025-03-03-15:02 | 0.2 |
| 4 | ... | ... | ... |
| 4 | 2025-03-03-14:30 | 2025-03-03-15:30 | 0.1 |
| 5 | 2025-03-03-14:30 | 2025-03-03-15:30 | `NaN` |

In [7]:
from glob import glob
from datetime import datetime


# set of sparse datetime points in the rain gauge dataset
T_ccrfcd = set([datetime.fromisoformat(_dt) for _dt in ckpt_one_df["utc_time"]])

# set of all unique datetimes in MRMS dataset
T_mrms = set()

all_mrms_data_dir = "__temp"
all_mrms_grib_files = glob(f"{all_mrms_data_dir}/*.grib2")

for fp in all_mrms_grib_files:
    dt_str = fp.split("_")[-1:][0].split(".")[0]
    dt = datetime.strptime(dt_str, "%Y%m%d-%H%M%S")
    T_mrms.add(dt)

##### 3.1 $\forall_{t \in \mathcal{T}_{MRMS}} \forall_{k \in \mathcal{I}} \exists $ *row in datatable*

From NSSL
> Product Creation
The one-hour QPE – Radar Only product is an aggregation of the Surface Precipitation Rate (SPR) field, which is updated every 2 minutes. The SPR fields from the previous 60 minutes are summed to create the QPE – Radar Only field, with the product ending at the indicated time. For instance, **a QPE – Radar Only field that is valid at 15:04Z is a summation of SPR fields from 14:04Z, 14:06Z, 14:08Z …to 15:04Z**. Values at or below 0.01 inches are removed to reduce the areal coverage of what is most likely false light precipitation.

In [8]:
import numpy as np
from datetime import timedelta


# set of unique rain gauge indicies
I = set(ckpt_one_df['gauge_idx'])

gauge_idxs = []
end_dts = []
start_dts = []


for dt in T_mrms:
    for gauge_idx in I:

        # MRMS 'valid time' is the end of a 1hr window
        end_dts.append(dt)
        
        start_dts.append(dt - timedelta(hours=1))
        gauge_idxs.append(gauge_idx)


dense_aligned_df = pd.DataFrame({
    'gauge_idx': gauge_idxs,
    'start_datetime_utc': start_dts,
    'end_datetime_utc': end_dts,
    'gauge_1h_acc': [np.nan] * len(end_dts)
})


dense_aligned_sorted_df = dense_aligned_df.sort_values(["start_datetime_utc", "gauge_idx"])
dense_aligned_sorted_df

,gauge_idx,start_datetime_utc,end_datetime_utc,gauge_1h_acc
301401,2.0,2021-01-23 23:00:00,2021-01-24 00:00:00,NaN
301421,4.0,2021-01-23 23:00:00,2021-01-24 00:00:00,NaN
301404,5.0,2021-01-23 23:00:00,2021-01-24 00:00:00,NaN
301409,7.0,2021-01-23 23:00:00,2021-01-24 00:00:00,NaN
301407,8.0,2021-01-23 23:00:00,2021-01-24 00:00:00,NaN
...,...,...,...,...
9957,5224.0,2025-08-13 11:46:00,2025-08-13 12:46:00,NaN
9964,5234.0,2025-08-13 11:46:00,2025-08-13 12:46:00,NaN
9974,5244.0,2025-08-13 11:46:00,2025-08-13 12:46:00,NaN
9991,5274.0,2025-08-13 11:46:00,2025-08-13 12:46:00,NaN


- we have: constructed a table with every unique (rain gauge, MRMS-timestep) combo
- want to: derive the 1hr accum precipitation for every row using the `ckpt_one` table

- $\mathcal{G}$: dataset of raw, CCRFCD rain gauge tips
    - $\mathcal{G}_{k} = \{(x_1, u_1), ..., (x_n, u_n) \}$
- $\mathcal{D}$: dense dataset containing MRMS timestamps that we want to populate
    - $\mathcal{D}_{k, t, t'} = \sum_{i}^m \delta x_i $ 
    - where $u_i \ge t$ and $u_m \le t'$

- > we can calculate the diffs for an entire gauge easily upfront

In [9]:
t1 = pd.to_datetime(pd.Timestamp("2021-08-01 22:26:00"), utc=True)
t2 = pd.to_datetime(pd.Timestamp("2021-08-01 23:26:00"), utc=True)

ckpt_one_df.loc[ckpt_one_df['gauge_idx'] == 4324.][:t1]

TypeError: cannot do slice indexing on Index with these indexers [2021-08-01 22:26:00+00:00] of type Timestamp

In [17]:
from zoneinfo import ZoneInfo

# we now have a table with blank entries for all unqiue (rain_gauge, MRMS-timestamp) combinations
# how can calculate the 1hr precipitation accumulation values for each of these entries?
# let's work through an example

# how much rain fell on gauge (4) between june and july of 2025?
t1 = datetime(year=2025, month=6, day=1).astimezone(ZoneInfo("UTC"))
t2 = datetime(year=2025, month=7, day=31).astimezone(ZoneInfo("UTC"))

subset = ckpt_one_df.copy()
subset = subset.set_index(pd.to_datetime(ckpt_one_df['utc_time']))
subset = subset[subset["gauge_idx"] == 4.0]

# element wise diffs; append a nan value to the start to match # rows
subset = subset.sort_index()
subset['gauge_acc_in_diff'] = [np.nan] + list(np.diff(subset['gauge_acc_in']))

start = pd.to_datetime(t1)
end   = pd.to_datetime(t2)

subset[start:end]['gauge_acc_in_diff'].sum()

np.float64(0.040000000000000036)

In [ ]:
# dataframe we constructed in 1.; contains raw gauge precip. accumulation values
ckpt_one_df_idxd = ckpt_one_df.set_index(pd.to_datetime(ckpt_one_df['utc_time'])).sort_index()

dense_aligned_sorted_df_clone = dense_aligned_sorted_df.copy().sort_values("start_datetime_utc")

In [55]:
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")


gauge_1hr_accums = []


# NOTE: think carefully through this section...
for i, row in tqdm(
    enumerate(dense_aligned_sorted_df_clone.itertuples()),
    total=len(dense_aligned_sorted_df_clone),
):

    # convert -> pd.Timestamp
    _t1 = pd.to_datetime(row.start_datetime_utc, utc=True)
    _t2 = pd.to_datetime(row.end_datetime_utc, utc=True)
    assert _t1 < _t2

    # limit subset to values for the current row's rain gauge
    gauge_all_df = ckpt_one_df_idxd.loc[ckpt_one_df_idxd["gauge_idx"] == row.gauge_idx]
    gauge_all_df['diffs'] = [np.nan] + list(np.diff(gauge_all_df['gauge_acc_in']))

    # grab subset of rain gauge tips that occured within [_t1, _t2]
    gauge_1hr_df = gauge_all_df.loc[_t1:_t2]
    gauge_prev_df = gauge_all_df.loc[:t1]

    # if there are 0 data points prior to _t1, no way to compute diffs
    if len(gauge_prev_df) < 1:
        gauge_1hr_accums.append(np.nan)
        continue

    if len(gauge_1hr_df) == 0:
        gauge_1hr_accums.append(0.0)
        continue

    acc_precip = np.array(gauge_1hr_df['diffs']).clip(0).sum()
    if acc_precip > 1.0:
        break

    gauge_1hr_accums.append(acc_precip)

 30%|██▉       | 158191/532840 [04:27<10:32, 592.16it/s]


In [ ]:
dense_aligned_sorted_df_clone["gauge_acc_in"] = gauge_1hr_accums
dense_aligned_sorted_df_clone[['gauge_idx', 'start_datetime_utc', 'end_datetime_utc', 'gauge_acc_in']].dropna().to_csv("data/__checkpoints__/ds_ckpt_3.csv")

### 4. Append MRMS 1H QPE; $\forall_{t \in \mathcal{T}_{MRMS}} \forall_{k \in \mathcal{I}}$
---

- 4a. Get gauge location: $\mathcal{I}_k = (lat, lon)$
- 4b. Find nearest MRMS grid-cell to rain gauge $k' = (lat', lon') \approx (lat, lon)$
- 4c. Get $MRMS_{t, k'}$; append to aligned dataset $D$
    - $D_{t, k, mrms} = $ $ MRMS_{t, k'} \over{25.4} $
    - > *Note: we divide by 25.4 to convert MRMS data from millimeters to inches*

| gauge-idx | start-datetime-utc | end-datetime-utc | gauge-1h-acc | mrms-1h-qpe |
| :---: | :---: | :---: | :---: | :---: |
| 4 | 2025-03-03-14:00 | 2025-03-03-15:00 | 0.1 | 0.13 |
| 4 | 2025-03-03-14:02 | 2025-03-03-15:02 | 0.2 | 0.19 |
| 4 | ... | ... | ... | ... |
| 4 | 2025-03-03-14:30 | 2025-03-03-15:30 | 0.1 | 0.25 |
| 5 | 2025-03-03-14:30 | 2025-03-03-15:30 | `NaN` | 0.08 |

- > *Note: we should have `mrms-1h-acc` values for all rows in this dataset, but not necessarily `gauge-1h-acc` values*

In [86]:
ckpt_3_df = pd.read_csv("data/__checkpoints__/ds_ckpt_3.csv")[['gauge_idx', 'start_datetime_utc', 'end_datetime_utc', 'gauge_acc_in']]
ckpt_3_df

,gauge_idx,start_datetime_utc,end_datetime_utc,gauge_acc_in
0,2.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.0
1,4464.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.0
2,4474.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.0
3,4479.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.0
4,4484.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.0
...,...,...,...,...
473547,4094.0,2025-08-13 11:46:00+00:00,2025-08-13 12:46:00+00:00,0.0
473548,4099.0,2025-08-13 11:46:00+00:00,2025-08-13 12:46:00+00:00,0.0
473549,4104.0,2025-08-13 11:46:00+00:00,2025-08-13 12:46:00+00:00,0.0
473550,4049.0,2025-08-13 11:46:00+00:00,2025-08-13 12:46:00+00:00,0.0
